In [60]:
import pandas as pd
import datetime

source = 'child-mind-institute-problematic-internet-use/series_train.parquet/id=0a418b57/part-0.parquet'

df = pd.read_parquet(source)
display(df)

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,-0.075242,-0.256743,-0.973791,0.038081,-72.952141,0.0,5.0,4202.000000,51250000000000,2,4,-9.0
1,1,-0.265893,-0.270508,-0.765470,0.077430,-52.849220,0.0,0.5,4185.333496,51255000000000,2,4,-9.0
2,2,0.334517,-0.548602,-0.588596,0.039162,-44.118084,0.0,11.5,4185.500000,51260000000000,2,4,-9.0
3,3,0.000193,-0.021069,-0.999681,0.001450,-88.759613,0.0,0.0,4185.666504,51265000000000,2,4,-9.0
4,4,-0.000685,-0.020681,-0.997677,0.000491,-88.756958,0.0,8.5,4185.833496,51270000000000,2,4,-9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213418,213418,0.391111,0.658119,-0.511350,0.060158,-35.455124,0.0,22.0,3817.430664,38745000000000,1,4,25.0
213419,213419,0.551039,0.078267,-0.787324,0.094029,-52.621586,0.0,22.0,3817.430664,38750000000000,1,4,25.0
213420,213420,0.667007,-0.126560,-0.751950,0.013104,-47.961002,0.0,22.0,3817.430664,39015000000000,1,4,25.0
213421,213421,0.666465,-0.125810,-0.749048,0.010495,-47.928806,0.0,22.0,3817.430664,39020000000000,1,4,25.0


In [61]:
df.where(df['non-wear_flag'] == 0, inplace=True)



# Convert nanoseconds to seconds and extract hours, minutes, seconds
df['seconds'] = df['time_of_day'].astype(int) / 1e9

# Convert the seconds to time of day using datetime.time
df['time_of_day'] = df['seconds'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(seconds=x)).time())

df = df.drop(columns=['step', 'X', 'Y', 'Z', 'non-wear_flag', 'anglez', 'battery_voltage', 'seconds'])

In [63]:
df.describe()

,enmo,light,weekday,quarter,relative_date_PCIAT
count,213423.000000,213423.000000,213423.000000,213423.0,213423.000000
mean,0.056785,34.272129,3.992409,4.0,1.118324
std,0.138663,161.996353,1.926265,0.0,6.136262
min,0.000000,0.000000,1.000000,4.0,-9.000000
25%,0.006549,2.642857,2.000000,4.0,-4.000000
50%,0.018648,7.500000,4.000000,4.0,1.000000
75%,0.052288,16.416666,6.000000,4.0,6.000000
max,4.674658,2622.600098,7.000000,4.0,25.000000
